# Sentiment Analysis of Movie Reviews using deep learning

## Introduction:

Sentiment analysis of movie reviews involves employing natural language processing to discern the emotional tone expressed in critiques. By evaluating textual nuances, algorithms gauge audience sentiments, identifying praise, criticism, or ambivalence. This analytical approach not only aids filmmakers in gauging audience reception but also contributes to the broader field of opinion mining and sentiment classification.

## Dataset:

Sentiment Analysis of Movie reviews is done via a deep learning model which is trained and evaluated on a dataset containing labeled examples of reviews with corresponding label indicating whether the sentiment of review is Positive or Negative.

link to the dataset: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [10]:
#using python 3.10.1
import pandas as pd
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
import math
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from keras.models import load_model
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils import to_categorical
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import OneHotEncoder

Lets first load the dataset

In [22]:
df=pd.read_csv("IMDBDataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

The code below sets up functions for common text preprocessing tasks, such as removing stopwords and lemmatizing words.

These preprocessing steps are crucial for cleaning and transforming raw text data before using it for NLP tasks like sentiment analysis or text classification.

In [23]:
nltk.download('stopwords')
print(stopwords.words('english'))
stop_words = set(stopwords.words('english')) 

def normalize_text(text):
    text = text.lower()
    # get rid of urls
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # get rid of non words and extra spaces
    text = re.sub('\\W', ' ', text)
    text = re.sub('\n', '', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('^ ', '', text)
    text = re.sub(' $', '', text)
    return text

def remove_stop_words(sentence): 
  if isinstance(sentence, str):
    words = sentence.split() 
    filtered_words = [word for word in words if word not in stop_words] 
    return ' '.join(filtered_words)
  else:
    return ""

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    if isinstance(text, str):
        st = ""
        for w in w_tokenizer.tokenize(text):
            st = st + lemmatizer.lemmatize(w) + " "
        return st.strip()
    else:
        return ""


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


Lets call the function and pass the reviews through them to process our dataset.

In [25]:
df['review']=df['review'].apply(normalize_text)
df['review']=df['review'].apply(remove_stop_words)
df['review']=df['review'].apply(lemmatize_text)

Here is how the reviews look like now.

In [26]:
df.head()

,review,sentiment
0,one reviewer mentioned watching 1 oz episode h...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake think zombie ...,negative
4,petter mattei love time money visually stunnin...,positive


In [38]:
y[3]

array([1., 0.])

We also have to calculate the average length of each sentence in the dataset (This will be of help later!)

In [27]:
def Cavg(text):
    arr=[]
    for i in text:
        np.array(arr.append(len(i)))
    return(np.average(arr))
avg_len=Cavg(df['review'])
print(avg_len)

817.50434


In [28]:
max_word=5000 #the maximum number of words to keep, based on word frequency.
max_sequence_length =850 #the average length was 932 so we can round of to 950
tokenizer = Tokenizer(num_words=max_word, split=' ') 
tokenizer.fit_on_texts(df['review'].values)
X = tokenizer.texts_to_sequences(df['review'].values)
X= pad_sequences(X, maxlen=max_sequence_length)

After completing the preprocessing of the dataset, the next step is to start preparing for deep learning modelling

In [29]:
one = OneHotEncoder()
y = one.fit_transform(np.asarray(df['sentiment']).reshape(-1,1)).toarray()
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

Lets quickly take a look at how the data is looking after splitting for training and testing.

In [30]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(35000, 850)
(35000, 2)
(15000, 850)
(15000, 2)


The code give below defines a sequential neural network model for a binary classification task using Keras. It consists of an Embedding layer followed by two Bidirectional LSTM layers with dropout. The final layer is a Dense layer with softmax activation. The model is compiled using the Adam optimizer and categorical crossentropy loss, with accuracy as the evaluation metric.

In [33]:
# Model
model = Sequential()
model.add(Embedding(max_word, 850, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(64, dropout=0.5, return_sequences=True)))
model.add(Bidirectional(LSTM(32, dropout=0.5)))
model.add(Dense(2, activation='softmax'))
# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Finally lets train the model on our training dataset. (You can change the number of epochs based on your liking. Just remember the more epochs the more time will the training process take.)

In [35]:
history = model.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
1094/1094 [==============================] - 1669s 2s/step - loss: 0.3430 - accuracy: 0.8542 - val_loss: 0.2878 - val_accuracy: 0.8756
Epoch 2/2
1094/1094 [==============================] - 1580s 1s/step - loss: 0.2319 - accuracy: 0.9095 - val_loss: 0.2560 - val_accuracy: 0.8965


After training is complete we will test the model on testing dataset and generate the classification report to analyse the performance of our model.


In [36]:
y_pred=model.predict(X_test)
report = classification_report(y_test.argmax(axis=1), np.around(y_pred, decimals=0).argmax(axis=1))
print("\nClassification Report:")
print(report)

469/469 [==============================] - 140s 298ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      7411
           1       0.89      0.91      0.90      7589

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



Finally lets save the model and the tokenizer so that it can be used later for applications. (I didnt run the cells below to save space.)

In [ ]:
import pickle
# saving deep learning model
model.save("MovieSent.h5")
# saving tokenizer
with open('tokenizer_movie.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


After saving the model and tokenizer we can load using the following code

In [ ]:
with open('tokenizer_email.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
loaded_model=keras.models.load_model('"EmailClassifier.h5"')

Lets Do some testing on input given by us!

In [39]:
max_sequence_length = 850
label=['Negative', 'Positive']

In [40]:
test_txt=['Disappointing from start to finish, this film seems to have missed the mark entirely. The plot is convoluted, characters lack depth, and the dialogue feels forced. Despite a promising premise, it descends into predictability. Even the stellar cast cant salvage this cinematic misstep. Save your time and skip this one','An absolute triumph in cinema, this film is a masterpiece from beginning to end. The storytelling is captivating, weaving a narrative that keeps you on the edge of your seat. The characters are rich and well-developed, brought to life by an outstanding cast. Visually stunning with a soundtrack to match, its a cinematic gem that deserves all the accolades it receives.']
seq = tokenizer.texts_to_sequences(test_txt)
ans = pad_sequences(seq, maxlen=max_sequence_length)
preds = model.predict(ans)
for i in preds:
    print(label[np.around(i, decimals=0).argmax()])

1/1 [==============================] - 0s 125ms/step
Negative
Positive
